# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression


In [4]:
spark = SparkSession.builder.appName('LR').getOrCreate()


In [6]:
df = spark.read.csv('/content/drive/MyDrive/Spark with Python/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/customer_churn.csv', inferSchema=True, header=True)

In [7]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [10]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [11]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [16]:
output = assembler.transform(df)

In [17]:
final_data = output.select('features', 'churn')

In [18]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [19]:
lr_churn = LogisticRegression(labelCol='churn')

In [20]:
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
training_sum = fitted_churn_model.summary

In [22]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                632|                632|
|   mean|0.18829113924050633|0.14873417721518986|
| stddev| 0.3912541386350016| 0.3561080948041746|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [26]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.56943249980088...|[0.98974246788674...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.72919099980652...|[0.67462771795881...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.19061430894194...|[0.99446225248490...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.50761023807480...|[0.81870677417965...|       0.0|
|[28.0,11128.95,1....|    0|[4.00785775545778...|[0.98215205486816...|       0.0|
|[28.0,11245.38,0....|    0|[3.59178945835117...|[0.97318961058893...|       0.0|
|[29.0,5900.78,1.0...|    0|[3.89767474226339...|[0.98011442522843...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.59064879506539...|[0.93025732181496...|       0.0|
|[29.0,10203.18,1....|    0|[3.57490181004583...|[0.97274544616377...|       0.0|
|[29.0,11274.46,

In [33]:
churn_eval = BinaryClassificationEvaluator(labelCol='churn', rawPredictionCol='prediction')

In [34]:
AUC = churn_eval.evaluate(pred_and_labels.predictions)

In [35]:
AUC

0.7208384374574656

## **Predict On New Data**

In [36]:
final_lr_model = lr_churn.fit(final_data)

In [37]:
new_customers = spark.read.csv('/content/drive/MyDrive/Spark with Python/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/new_customers.csv', inferSchema=True, header=True)

In [38]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [39]:
test_new_customer = assembler.transform(new_customers)

In [40]:
test_new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [41]:
final_results = final_lr_model.transform(test_new_customer)

In [45]:
final_results.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

